In [1]:
%cd ..

/Users/lisaschmidt/Documents/GitHub/data-engineering-showcase


In [2]:
%cd /Users/lisaschmidt/Documents/GitHub/data-engineering-showcase

/Users/lisaschmidt/Documents/GitHub/data-engineering-showcase


# Dataset 1

### Downloading data

In [10]:
from urllib.request import urlretrieve

In [11]:
url = ("https://mobilithek.info/mdp-api/files/aux/573356838940979200/moin-2022-05-02.1-20220502.131229-1.ttl.bz2")
filename = "city_connections.ttl.bz2"

In [12]:
urlretrieve(url, filename)

('city_connections.ttl.bz2', <http.client.HTTPMessage at 0x10c64e3d0>)

In [13]:
import bz2

In [14]:
zipfile = bz2.BZ2File(filename) # open the file
data = zipfile.read() # get the decompressed data
newfilepath = filename[:-4] # assuming the filepath ends with .bz2
open(newfilepath, 'wb').write(data) # write a uncompressed file

167540483

In [15]:
import pyoxigraph as graph
import pandas as pd

In [16]:
l = list(graph.parse(newfilepath, "text/turtle", base_iri="http://example.com/"))

### An Overview of the Data Structure

In [17]:
for t in l[5:15]:
    print( t.subject, t.predicate, t.object)

<http://moin-project.org/data/Bremerhaven> <http://moin-project.org/ontology/connectedTo> <http://moin-project.org/data/Marl> <http://moin-project.org/ontology/hasTrip> _:b03efbe9650ea62787776fa20cb4760f
_:b03efbe9650ea62787776fa20cb4760f <http://moin-project.org/ontology/transportType> <http://moin-project.org/ontology/train>
_:b03efbe9650ea62787776fa20cb4760f <http://moin-project.org/ontology/startTime> "10:42:00"^^<http://www.w3.org/2001/XMLSchema#time>
_:b03efbe9650ea62787776fa20cb4760f <http://moin-project.org/ontology/endTime> "16:02:00"^^<http://www.w3.org/2001/XMLSchema#time>
_:b03efbe9650ea62787776fa20cb4760f <http://moin-project.org/ontology/duration> "PT320M"^^<http://www.w3.org/2001/XMLSchema#duration>
<http://moin-project.org/data/Bremerhaven> <http://moin-project.org/ontology/connectedTo> <http://moin-project.org/data/Marl> <http://moin-project.org/ontology/hasTrip> _:b8600e2631160ecfda276fd8dc63698b
_:b8600e2631160ecfda276fd8dc63698b <http://moin-project.org/ontology/tra

This graph has different types of triples that are useful to us:

1. A triple which describes that two cities are connected by a trip:

    ```<http://moin-project.org/data/Bremerhaven> <http://moin-project.org/ontology/connectedTo> <http://moin-project.org/data/Marl> <http://moin-project.org/ontology/hasTrip> _:77af2dc5c57b5384d50fd429a6ae23e4```
    
    Notable is here that the subject `<http://moin-project.org/data/Bremerhaven> <http://moin-project.org/ontology/connectedTo> <http://moin-project.org/data/Marl>` is a triple itself. This "city-connection" is then uniquely identified through `hasTrip` to an identifier `_:77af2dc5c57b5384d50fd429a6ae23e4`.

2. A triple which describes the properties of a trip (ex. transport type, travel time or distance):

    ```_:77af2dc5c57b5384d50fd429a6ae23e4 <http://moin-project.org/ontology/transportType> <http://moin-project.org/ontology/train>```

In [6]:
# a connection between two cities
l[0].subject

<Triple subject=<NamedNode value=http://moin-project.org/data/Bremerhaven> predicate=<NamedNode value=http://moin-project.org/ontology/connectedTo> object=<NamedNode value=http://moin-project.org/data/Marl>>

In [7]:
# an example of a triple which states a connection between two cities and then uniquely names them.
l[0]

<Triple subject=<Triple subject=<NamedNode value=http://moin-project.org/data/Bremerhaven> predicate=<NamedNode value=http://moin-project.org/ontology/connectedTo> object=<NamedNode value=http://moin-project.org/data/Marl>> predicate=<NamedNode value=http://moin-project.org/ontology/hasTrip> object=<BlankNode value=efe3ea1fdf24bf4991ed685c632596bb>>

In [8]:
print("There are ", len(l), " triples in the database.")

There are  215136  triples in the database.


In [9]:
connections = [t.subject for t in l]
for i in range(20):
    print(i, connections[i])

0 <http://moin-project.org/data/Bremerhaven> <http://moin-project.org/ontology/connectedTo> <http://moin-project.org/data/Marl>
1 _:efe3ea1fdf24bf4991ed685c632596bb
2 _:efe3ea1fdf24bf4991ed685c632596bb
3 _:efe3ea1fdf24bf4991ed685c632596bb
4 _:efe3ea1fdf24bf4991ed685c632596bb
5 <http://moin-project.org/data/Bremerhaven> <http://moin-project.org/ontology/connectedTo> <http://moin-project.org/data/Marl>
6 _:b46bf39bba0d7689b9b9f928de67bd21
7 _:b46bf39bba0d7689b9b9f928de67bd21
8 _:b46bf39bba0d7689b9b9f928de67bd21
9 _:b46bf39bba0d7689b9b9f928de67bd21
10 <http://moin-project.org/data/Bremerhaven> <http://moin-project.org/ontology/connectedTo> <http://moin-project.org/data/Marl>
11 _:5fbb45c6dd306cf40e36840736a0290
12 _:5fbb45c6dd306cf40e36840736a0290
13 _:5fbb45c6dd306cf40e36840736a0290
14 _:5fbb45c6dd306cf40e36840736a0290
15 <http://moin-project.org/data/Bremerhaven> <http://moin-project.org/ontology/connectedTo> <http://moin-project.org/data/Marl>
16 _:5dcf820dd3623d4a7faecafe21ad8906
17 _

We can see above that there are different options to travel between Bremerhaven and Marl. In this project, we want to compare these different travel option with a special interest in comparing travel by car to travel by train.

### Overview on the Contents of the Data

In [10]:
predicates = [t.predicate for t in l]
pd.DataFrame(predicates).value_counts()

<http://moin-project.org/ontology/duration>               40515
<http://moin-project.org/ontology/hasTrip>                40515
<http://moin-project.org/ontology/transportType>          40515
<http://moin-project.org/ontology/startTime>              30524
<http://moin-project.org/ontology/endTime>                30461
<http://moin-project.org/ontology/connectedTo>            10159
<http://moin-project.org/ontology/route>                   9991
<http://moin-project.org/ontology/drivingDistance>         9991
<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>           420
<http://www.w3.org/2000/01/rdf-schema#label>                233
<http://www.wikidata.org/prop/direct/P625>                  226
<http://www.wikidata.org/prop/direct/P17>                   117
<http://www.wikidata.org/prop/direct/P31>                   117
<http://www.wikidata.org/prop/direct/P239>                  117
<http://www.opengis.net/ont/geosparql#asWKT>                103
<http://www.opengis.net/ont/geosparql#de

The most interesting predicates for this project are the `moin-project` predicates:
- duration
- hasTrip
- transportType
- connectedTo
- route

The other `moin-project` predicates (startTime, endTime, drivingDistance, nearestAirport) are currently not relevant.

But there are other predicates, which we also should investigate. For example, the `wikidata` predicates:

- P625 = Coordinate Information
- P15 = country
- P31 = instance of
- P239 ICAO Airport Code
- P94 = Coat of arms image
- P131 = located in the administrative territorial entity
- P1082 = Population
- P856 = official website
- P2046 = area
- P41 = flag image
- P238 = IATA Airport Code

##### Now let's take a look at the remaining predicates:

- <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> 
- <http://www.w3.org/2000/01/rdf-schema#label>   
- <http://www.w3.org/2002/07/owl#sameAs>
- <http://www.w3.org/2000/01/rdf-schema#comment>


- <http://schema.org/about>


- <http://www.opengis.net/ont/geosparql#asWKT>            
- <http://www.opengis.net/ont/geosparql#defaultGeometry>  

In [11]:
from pyoxigraph import Store

In [12]:
store = Store()
store.load('./data/moin-2022-05-02.1-20220502.131229-1.ttl', mime_type="text/turtle")

In [53]:
PREDICATE = "http://www.opengis.net/ont/geosparql#defaultGeometry"

result = store.query("SELECT ?s ?o WHERE { ?s <"+PREDICATE+"> ?o}")

In [54]:
[s for s in result][:10]

[<QuerySolution s=<NamedNode value=http://moin-project.org/data/Bonn> o=<BlankNode value=178d5895dd965984742aceab83deffb>>,
 <QuerySolution s=<NamedNode value=http://moin-project.org/data/Hanau> o=<BlankNode value=2940e5c922713208c279eeb70e08011>>,
 <QuerySolution s=<NamedNode value=http://moin-project.org/data/Gelsenkirchen> o=<BlankNode value=2973a0274bb3c7580447c0395ccf5f0>>,
 <QuerySolution s=<NamedNode value=http://moin-project.org/data/Braunschweig> o=<BlankNode value=2b04f868e9e89d5a875c00c63f65852>>,
 <QuerySolution s=<NamedNode value=http://moin-project.org/data/Ludwigsburg> o=<BlankNode value=3570c5b7c56ec43b68050933d6c1d21>>,
 <QuerySolution s=<NamedNode value=http://moin-project.org/data/Heilbronn> o=<BlankNode value=48ffed53900dba8101f63ef7448a065>>,
 <QuerySolution s=<NamedNode value=http://moin-project.org/data/Recklinghausen> o=<BlankNode value=4a60fd556596b23b3e08d764a4a2cad>>,
 <QuerySolution s=<NamedNode value=http://moin-project.org/data/Dessau-Ro%C3%9Flau> o=<Blank

- <http://www.w3.org/1999/02/22-rdf-syntax-ns#type>     
    The type of an object, for example if it is an Airport
- <http://www.w3.org/2000/01/rdf-schema#label>   
    This predicate relates an IRI to a literal, for example the IRI of a city to the city's name.
- <http://www.w3.org/2002/07/owl#sameAs>    
    The different IRIs of the same object are related through this predicate.
- <http://www.w3.org/2000/01/rdf-schema#comment>    
    The comment field contains a short description of different cities.
- <http://schema.org/about>      
    This relation links a city's wikipedia article to the IRI.
- <http://www.opengis.net/ont/geosparql#asWKT>      
    The WKT-Literal contains the coordinates of an object.        
- <http://www.opengis.net/ont/geosparql#defaultGeometry>    
    This predicate connects a city's IRI to an object (see above).

## A Plan for bringing the Data into the desired Format

### Requirements:
- One Table with all **trips as primary key**
- Columns: 
    - duration
    - hasTrip
    - transportType
    - connectedTo
    - route

### Approach
[ ] Read all trips into a structure and match them with start IRI and destination IRI

[ ] Find all trip properties by the trip IRI and write them to the trips  

[ ] Find the labels of start and end destination and replace IRIs with labels 

[ ] 

## Data Transformations

In [15]:
type(l[0].subject.predicate)

pyoxigraph.NamedNode

In [16]:
connected_to = graph.NamedNode("http://moin-project.org/ontology/connectedTo")
has_trip = graph.NamedNode("http://moin-project.org/ontology/hasTrip")

In [72]:
#PREDICATE = "http://moin-project.org/ontology/connectedTo"
PREDICATE = "http://moin-project.org/ontology/hasTrip"

result = store.query("SELECT ?s ?o WHERE { ?s <"+PREDICATE+"> ?o}")

#### (1) Read all trips into a structure and match them with start IRI and destination IRI

In [165]:
connections = {}
for triple in l:
    try:
        subject, predicate, object = triple.subject, triple.predicate, triple.object
        if predicate == has_trip: #subject.predicate == connected_to and 
            trip_from = subject.subject
            trip_to = subject.object
            trip_id = triple.object
            connections[trip_id.value] = {
                "iri_start": str(trip_from.value),
                "iri_end": str(trip_to.value),
            }
    except AttributeError:
        print("Error with triple: ", triple)

In [166]:
df_connections = pd.DataFrame(connections).T
df_connections

,iri_start,iri_end
efe3ea1fdf24bf4991ed685c632596bb,http://moin-project.org/data/Bremerhaven,http://moin-project.org/data/Marl
b46bf39bba0d7689b9b9f928de67bd21,http://moin-project.org/data/Bremerhaven,http://moin-project.org/data/Marl
5fbb45c6dd306cf40e36840736a0290,http://moin-project.org/data/Bremerhaven,http://moin-project.org/data/Marl
5dcf820dd3623d4a7faecafe21ad8906,http://moin-project.org/data/Bremerhaven,http://moin-project.org/data/Marl
3757e368a3098d96a56ddd3680e31819,http://moin-project.org/data/Dortmund,http://moin-project.org/data/Karlsruhe
...,...,...
658c7cb4cf9c190c448c9b0f154e288a,http://moin-project.org/data/Chemnitz,http://moin-project.org/data/Osnabr%C3%BCck
4b8856e43c21db8e35cd7ac186d145d,http://moin-project.org/data/Halle%20%28Saale%29,http://moin-project.org/data/Bielefeld
909079ff4d7306729d009a6acf72d453,http://moin-project.org/data/Halle%20%28Saale%29,http://moin-project.org/data/Bielefeld
842686b28b3d3340abc3d38d0ba156c5,http://moin-project.org/data/Halle%20%28Saale%29,http://moin-project.org/data/Bielefeld


#### (2) Find all trip properties by the trip IRI and write them to the trips  

In [167]:
duration = graph.NamedNode("http://moin-project.org/ontology/duration")
transport_type = graph.NamedNode("http://moin-project.org/ontology/transportType")
route = graph.NamedNode("http://moin-project.org/ontology/route")

In [168]:
for triple in l:
    try:
        subject, predicate, object = triple.subject, triple.predicate, triple.object
        pred_name = None
        pred_value = None
        if predicate in [duration, transport_type, route]:
            trip_id = triple.subject.value
            pred_name = triple.predicate.value
            pred_value = triple.object.value
            connections[trip_id][str(pred_name)] = str(pred_value)
    except AttributeError:
        print("Error with triple: ", triple)

In [169]:
df_connections = pd.DataFrame(connections).T
df_connections

,iri_start,iri_end,http://moin-project.org/ontology/transportType,http://moin-project.org/ontology/route,http://moin-project.org/ontology/duration
efe3ea1fdf24bf4991ed685c632596bb,http://moin-project.org/data/Bremerhaven,http://moin-project.org/data/Marl,http://moin-project.org/ontology/car,LINESTRING(8.586580000000001 53.55175000000000...,PT9134.0S
b46bf39bba0d7689b9b9f928de67bd21,http://moin-project.org/data/Bremerhaven,http://moin-project.org/data/Marl,http://moin-project.org/ontology/train,NaN,PT320M
5fbb45c6dd306cf40e36840736a0290,http://moin-project.org/data/Bremerhaven,http://moin-project.org/data/Marl,http://moin-project.org/ontology/train,NaN,PT385M
5dcf820dd3623d4a7faecafe21ad8906,http://moin-project.org/data/Bremerhaven,http://moin-project.org/data/Marl,http://moin-project.org/ontology/train,NaN,PT322M
3757e368a3098d96a56ddd3680e31819,http://moin-project.org/data/Dortmund,http://moin-project.org/data/Karlsruhe,http://moin-project.org/ontology/car,"LINESTRING(7.46417 51.51505, 7.461770000000000...",PT12015.0S
...,...,...,...,...,...
658c7cb4cf9c190c448c9b0f154e288a,http://moin-project.org/data/Chemnitz,http://moin-project.org/data/Osnabr%C3%BCck,http://moin-project.org/ontology/train,NaN,PT320M
4b8856e43c21db8e35cd7ac186d145d,http://moin-project.org/data/Halle%20%28Saale%29,http://moin-project.org/data/Bielefeld,http://moin-project.org/ontology/car,LINESTRING(11.970030000000001 51.4824400000000...,PT10854.0S
909079ff4d7306729d009a6acf72d453,http://moin-project.org/data/Halle%20%28Saale%29,http://moin-project.org/data/Bielefeld,http://moin-project.org/ontology/train,NaN,PT231M
842686b28b3d3340abc3d38d0ba156c5,http://moin-project.org/data/Halle%20%28Saale%29,http://moin-project.org/data/Bielefeld,http://moin-project.org/ontology/train,NaN,PT192M


In [170]:
df_connections["http://moin-project.org/ontology/transportType"].value_counts()

http://moin-project.org/ontology/transportType
http://moin-project.org/ontology/train     29106
http://moin-project.org/ontology/car        9991
http://moin-project.org/ontology/flight     1418
Name: count, dtype: int64

#### (3) Find the labels of start and end destination and replace IRIs with labels 

In [171]:
PREDICATE = "http://www.w3.org/2000/01/rdf-schema#label"

In [172]:
IRI2Label = {}

In [173]:
result = store.query("SELECT ?subject ?object WHERE { ?subject <"+PREDICATE+"> ?object}")

for r in result:
    IRI2Label[str(r["subject"].value)] = r["object"].value
    #print(r["subject"].value)
    #print(r["object"].value)

In [176]:
def replace_with_label(iri):
    if iri in list(IRI2Label.keys()):
        return IRI2Label[iri]

In [178]:
df_connections["iri_start"].apply(replace_with_label)

efe3ea1fdf24bf4991ed685c632596bb      Bremerhaven
b46bf39bba0d7689b9b9f928de67bd21      Bremerhaven
5fbb45c6dd306cf40e36840736a0290       Bremerhaven
5dcf820dd3623d4a7faecafe21ad8906      Bremerhaven
3757e368a3098d96a56ddd3680e31819         Dortmund
                                        ...      
658c7cb4cf9c190c448c9b0f154e288a         Chemnitz
4b8856e43c21db8e35cd7ac186d145d     Halle (Saale)
909079ff4d7306729d009a6acf72d453    Halle (Saale)
842686b28b3d3340abc3d38d0ba156c5    Halle (Saale)
7ff289ce10418a0482a5f7437795c680    Halle (Saale)
Name: iri_start, Length: 40515, dtype: object

In [179]:
df_connections["iri_end"].apply(replace_with_label)

efe3ea1fdf24bf4991ed685c632596bb         Marl
b46bf39bba0d7689b9b9f928de67bd21         Marl
5fbb45c6dd306cf40e36840736a0290          Marl
5dcf820dd3623d4a7faecafe21ad8906         Marl
3757e368a3098d96a56ddd3680e31819    Karlsruhe
                                      ...    
658c7cb4cf9c190c448c9b0f154e288a    Osnabrück
4b8856e43c21db8e35cd7ac186d145d     Bielefeld
909079ff4d7306729d009a6acf72d453    Bielefeld
842686b28b3d3340abc3d38d0ba156c5    Bielefeld
7ff289ce10418a0482a5f7437795c680    Bielefeld
Name: iri_end, Length: 40515, dtype: object

In [180]:
def replace_transport_iri_with_label(transport_type_iri):
    if transport_type_iri == "http://moin-project.org/ontology/train":
        return "train"
    if transport_type_iri == "http://moin-project.org/ontology/car":
        return "car"
    if transport_type_iri == "http://moin-project.org/ontology/flight":
        return "flight"

In [183]:
df_connections["http://moin-project.org/ontology/transportType"].apply(replace_transport_iri_with_label)

efe3ea1fdf24bf4991ed685c632596bb      car
b46bf39bba0d7689b9b9f928de67bd21    train
5fbb45c6dd306cf40e36840736a0290     train
5dcf820dd3623d4a7faecafe21ad8906    train
3757e368a3098d96a56ddd3680e31819      car
                                    ...  
658c7cb4cf9c190c448c9b0f154e288a    train
4b8856e43c21db8e35cd7ac186d145d       car
909079ff4d7306729d009a6acf72d453    train
842686b28b3d3340abc3d38d0ba156c5    train
7ff289ce10418a0482a5f7437795c680    train
Name: http://moin-project.org/ontology/transportType, Length: 40515, dtype: object